In [1]:
%matplotlib inline

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import numpy as np
import pandas as pd
import warnings

from matplotlib import pyplot as plt

from IPython.core.pylabtools import figsize
figsize(11, 9)

import collections

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.python import debug as tf_debug

tfd = tfp.distributions

/home/aapopovkin/venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/aapopovkin/venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/aapopovkin/venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/aapopovkin/venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/aapopovkin/venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import tensorflow.contrib.eager as tfe
from tensorflow.python.eager.context import eager_mode, graph_mode

In [3]:
import pandas as pd
import numpy as np

In [4]:
# Handy snippet to reset the global graph and global session.
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    tf.reset_default_graph()
    try:
        sess.close()
    except:
        pass
    sess = tf.InteractiveSession()
    
    #sess = tf_debug.LocalCLIDebugWrapperSession(sess)

In [5]:
inv_alpha_transform = lambda y: np.log(y)  # Not using TF here.
fwd_alpha_transform = tf.exp

This is model, where we should do binary classification - predict probability of A
   1) num_classes - number of classes
   2) each class has its popability of A
   3) prob - array of these probabilities; they are Q - coordinates for Hamiltonian MCMC
   4) prior (alpha, beta) are two float variables describing beta-distribution classes' popabilities.

In [6]:
def _make_ps_prior(num_classes, dtype):
    raw_prior_alpha = tf.get_variable(
      name='raw_prior_alpha',
      initializer=np.array(inv_alpha_transform(5.), dtype=dtype))
    raw_prior_beta = tf.get_variable(
      name='raw_prior_beta',
      initializer=np.array(inv_alpha_transform(5.), dtype=dtype))   
    return tfp.distributions.Independent(
      tfp.distributions.Beta(
          fwd_alpha_transform(raw_prior_alpha) * tf.ones(num_classes),
          fwd_alpha_transform(raw_prior_beta) * tf.ones(num_classes)),
      reinterpreted_batch_ndims=1)

make_ps_prior = tf.make_template(name_='make_ps_prior', func_=_make_ps_prior)

In [7]:
beta3_5 = tfp.distributions.Beta(3,5)

In [8]:
beta3_5.sample()

<tf.Tensor 'Beta/sample/Reshape:0' shape=() dtype=float32>

In [9]:
beta3_5 = tfp.distributions.Beta([3,3],[5,5])

In [10]:
beta3_5.sample()
sess.run(beta3_5.sample())

array([0.33741772, 0.34241349], dtype=float32)

In [11]:
sess.run(5 * tf.ones(10))

array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5.], dtype=float32)

In [12]:
two_betas = tfp.distributions.Independent(tfp.distributions.Beta(3 * tf.ones(2), 5 * tf.ones(2)),  reinterpreted_batch_ndims=1)

In [13]:
sess.run(two_betas.sample())

array([0.5657742 , 0.52742934], dtype=float32)

In [14]:
d = tfp.distributions.Binomial(total_count=[5.0,5.0,5.0], probs=[0.3,0.4,0.99])
print(d)
print(sess.run(d.prob([1,2,5])))

tfp.distributions.Binomial("Binomial/", batch_shape=(3,), event_shape=(), dtype=float32)
[0.36014998 0.34559998 0.9509907 ]


In [15]:
d = tfp.distributions.Binomial(total_count=5.0, probs=[0.3,0.4,0.99])
print(d)
print(sess.run(tf.reduce_sum(d.log_prob([1,2,5]), axis=-1)))

tfp.distributions.Binomial("Binomial_1/", batch_shape=(3,), event_shape=(), dtype=float32)
-2.133959


In [16]:
def _make_ps_log_likelihood(prob, class_id, total_count):
    prob_c = tf.gather(prob, indices=tf.to_int32(class_id - 1), axis=-1)
    total_count_c = tf.gather(total_count, indices=tf.to_int32(class_id - 1), axis=-1)
    return tfp.distributions.Binomial(total_count=tf.to_float(total_count_c), probs=prob_c)

make_ps_log_likelihood = tf.make_template(name_='make_ps_log_likelihood', func_=_make_ps_log_likelihood)

In [17]:
res = tf.gather([0.1,0.2,0.3,0.4,0.5,0.6], indices=tf.to_int32([0,0,4,4,1,1,2,2,3,3]))
print(sess.run(res)+1)

Instructions for updating:
Use tf.cast instead.
[1.1 1.1 1.5 1.5 1.2 1.2 1.3 1.3 1.4 1.4]


In [18]:
def joint_log_prob(prob, total_count, clicks, class_id, dtype):
    num_classes = len(total_count)
    rv_prob = make_ps_prior(num_classes, dtype)
    rv_clicks = make_ps_log_likelihood(prob, class_id, total_count)
    return (rv_prob.log_prob(prob) + 
        tf.reduce_sum(rv_clicks.log_prob(clicks), axis=-1))

In [19]:
ps_data_arr = np.array([
    [20, 10, 1, 0.499993], [20, 3, 2, 0.230211], [20, 8, 3, 0.236831], 
    [20, 7, 4, 0.246463], [20, 6, 5, 0.370862], [20, 5, 6, 0.320656], 
    [20, 10, 7, 0.519887], [20, 12, 8, 0.52845], [20, 8, 9, 0.453077], 
    [20, 8, 10, 0.431245], [20, 10, 11, 0.499243], [20, 9, 12, 0.471968], 
    [20, 2, 13, 0.152176], [20, 14, 14, 0.48496], [20, 6, 15, 0.246193]
    ])

In [20]:
ps_data_pd=pd.DataFrame(data=ps_data_arr[0:, 0:],
             index=ps_data_arr[0:, 2],
             columns=["total_count", "clicks", "class_id", "true_p"],
             dtype=np.float32)

In [21]:
ps_data_pd['class_id'] = ps_data_pd.class_id.astype('int32')

In [22]:
ps_data_pd.dtypes

total_count    float32
clicks         float32
class_id         int32
true_p         float32
dtype: object

In [23]:
ps_data_pd.drop_duplicates()

,total_count,clicks,class_id,true_p
1.0,20.0,10.0,1,0.499993
2.0,20.0,3.0,2,0.230211
3.0,20.0,8.0,3,0.236831
4.0,20.0,7.0,4,0.246463
5.0,20.0,6.0,5,0.370862
6.0,20.0,5.0,6,0.320656
7.0,20.0,10.0,7,0.519887
8.0,20.0,12.0,8,0.528450
9.0,20.0,8.0,9,0.453077
10.0,20.0,8.0,10,0.431245


In [24]:
# Specify unnormalized posterior.

dtype = np.float32


def unnormalized_posterior_log_prob(prob):
    return joint_log_prob(
        prob=prob,
        total_count=dtype(ps_data_pd.total_count.values),
        clicks=dtype(ps_data_pd.clicks.values),
        class_id=np.int32(ps_data_pd.class_id.values),
        dtype=dtype)


In [25]:
# Set-up E-step.
'''
def get_step_size():
    with tf.variable_scope("foo", reuse=tf.AUTO_REUSE):
        tf.get_variable(
            'step_size',
            initializer=np.array(0.2, dtype=dtype),
            trainable=False)

step_size = get_step_size()
''';

In [26]:
# Set-up E-step.
step_size = tf.get_variable(
            'step_size',
            initializer=np.array(0.1, dtype=dtype),
            trainable=False)

Instructions for updating:
Colocations handled automatically by placer.


In [119]:
hmc = tfp.mcmc.HamiltonianMonteCarlo(
    target_log_prob_fn=unnormalized_posterior_log_prob,
    num_leapfrog_steps=10,
    step_size=0.01,#step_size,
    #step_size_update_fn=tfp.mcmc.make_simple_step_size_update_policy(
    #  num_adaptation_steps=None),
    state_gradients_are_stopped=True)

init_random_weights = tf.placeholder(dtype, shape=[len(ps_data_pd)])

posterior_random_weights, kernel_results = tfp.mcmc.sample_chain(
    num_results=3,
    num_burnin_steps=0,
    num_steps_between_results=0,
    current_state=init_random_weights,
    kernel=hmc)

In [120]:
# Initialize all variables.

init_op = tf.global_variables_initializer()

In [121]:
init_op.run()
w_ = np.random.beta(1, 2, size=len(ps_data_pd)).astype(dtype)# 0.5 * np.ones([len(ps_data_pd)], dtype=dtype)

In [137]:
print(sess.run([
    kernel_results,
    #posterior_random_weights,
    #loss,
    #train_op
], feed_dict={init_random_weights: w_}))

[MetropolisHastingsKernelResults(accepted_results=UncalibratedHamiltonianMonteCarloKernelResults(log_acceptance_correction=array([-95.56643  ,  -2.0686707,   4.724431 ], dtype=float32), target_log_prob=array([-26.601244, -24.530464, -29.267681], dtype=float32), grads_target_log_prob=[array([[-15.197369 ,   7.0576315,  -4.719725 ,   2.2450454, -11.223586 ,
          9.550534 , -14.106207 ,   0.8620472,  19.21813  , -24.370537 ,
        -15.544897 ,  -6.2250757,  -1.3335171,   7.8560624,   7.7854166],
       [ -9.514488 ,  29.795773 ,   6.4243174,  10.657215 ,  -9.106619 ,
         -8.541821 , -10.163707 , -10.200941 ,  13.774683 ,  -2.9328756,
        -13.30499  ,  -5.376053 ,  15.151863 ,  -1.8187151,   1.2874784],
       [-21.663862 ,   1.1247444,  -1.1662779,   8.142054 ,  11.235966 ,
        -11.999753 ,   1.2397634, -13.92033  ,  10.121515 ,  -9.391367 ,
        -36.74359  ,  -3.6788948,  -4.3266897,  19.055866 ,  12.37598  ]],
      dtype=float32)]), is_accepted=array([ True,  Tru

In [138]:
# Set-up M-step.

loss = -tf.reduce_mean(kernel_results.accepted_results.target_log_prob)

global_step = tf.train.get_or_create_global_step()

learning_rate = tf.train.exponential_decay(
    learning_rate=0.1,
    global_step=global_step,
    decay_steps=2,
    decay_rate=0.99)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss, global_step=global_step)

ValueError: Variable make_ps_prior/raw_prior_alpha/Adam/ already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-28-b77bf5c240ab>", line 14, in <module>
    train_op = optimizer.minimize(loss, global_step=global_step)
  File "/home/aapopovkin/venv/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/aapopovkin/venv/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2903, in run_ast_nodes
    if self.run_code(code, result):


In [ ]:
# Grab variable handles for diagnostic purposes.

with tf.variable_scope('make_ps_prior', reuse=True):
    prior_alpha = fwd_alpha_transform(tf.get_variable(
        name='raw_prior_alpha', dtype=dtype))
    prior_beta = fwd_alpha_transform(tf.get_variable(
        name='raw_prior_beta', dtype=dtype))

In [32]:
print("{:.4f}".format(sess.run(prior_beta)))

5.0000


In [33]:
sess.run([
    unnormalized_posterior_log_prob(w_),
], feed_dict={init_random_weights: w_})

[-148.47824]

In [131]:
maxiter = int(50)
num_accepted = 0
num_drawn = 0
for i in range(1):
    posterior_random_weights_,
    kernel_results_,
    loss_,
    train_or = sess.run([
        kernel_results,
        posterior_random_weights,
        loss,
        train_op
    ], feed_dict={init_random_weights: w_})
    
    global_step_ = sess.run(global_step)
    step_size_ = sess.run(step_size)
    prior_alpha_ = sess.run(prior_alpha)
    prior_beta_ = sess.run(prior_beta)
    
    w_ = posterior_random_weights_[-1, :]
    num_accepted += kernel_results_.is_accepted.sum()
    num_drawn += kernel_results_.is_accepted.size
    acceptance_rate = num_accepted / num_drawn
    if i % 1 == 0 or i == maxiter - 1:
        print('global_step:{:>4}  loss:{: 9.3f}  acceptance:{:.4f}  '
              'step_size:{:.4f}  prior_alpha:{:.4f}  prior_beta:{:.4f}'.format(
                  global_step_, loss_.mean(),
                  acceptance_rate, step_size_,
                  prior_alpha_, prior_beta_)
        )

global_step:   1  loss:      nan  acceptance:0.0000  step_size:0.1941  prior_alpha:nan  prior_beta:nan
